In [34]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [35]:
froot = "C:/Users/verdi/Documents/Datasets/IEEEFraudDetection/ieee-fraud-detection/"
transaction_fname = "train_transaction.csv"
id_fname = "train_identity.csv"
transaction_df = pd.read_csv(froot + transaction_fname)
identity_df = pd.read_csv(froot + id_fname)
df = transaction_df.merge(identity_df, how="left", on="TransactionID")
df.head(5)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


## Data types

In [36]:
num_df = df.select_dtypes(include=np.number)
cat_df = df.select_dtypes(exclude=np.number)
cat_cols = cat_df.columns
num_cols = num_df.columns

print("Number of numerical columns: %d" % len(num_cols))
print("Number of categorical columns: %d" % len(cat_cols))
print("Categorical columns: {}".format(cat_cols))

Number of numerical columns: 403
Number of categorical columns: 31
Categorical columns: Index(['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1',
       'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15',
       'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33',
       'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo'],
      dtype='object')


## Invalid values
- Numerical attributes
- Categorical attributes

In [37]:
# num_nan_cols = df.columns[df.isna().any()]
# nan_ratio = len(num_nan_cols) / len(num_cols)
# print("There are %d numerical columns with missing values (%2.4f of numerical columns)" % (len(num_nan_cols), nan_ratio))

# nan_cols_sum_series = df.isna().sum().sort_values(ascending=False)
# nan_cols_ratio_series = (df.isna().sum() / len(df)).sort_values(ascending=False)
# nan_cols_df = pd.DataFrame(
#     pd.concat((nan_cols_sum_series, nan_cols_ratio_series), axis=1),
# )
# nan_cols_df.style.to_latex("export/table/nan_col_series.tex")
# nan_cols_df.head(20)

In [38]:
nan_cols = df.columns[df.isna().any()]
nan_ratio = len(nan_cols) / len(df.columns)
print("There are %d columns with missing values (%2.4f of all columns)" % (len(nan_cols), nan_ratio))

nan_cols_sum_series = df.isna().sum().sort_values(ascending=False)
nan_cols_ratio_series = (df.isna().sum() / len(df)).sort_values(ascending=False)
nan_cols_df = pd.DataFrame(
    pd.concat((nan_cols_sum_series, nan_cols_ratio_series), axis=1),
)
nan_cols_df.style.to_latex("export/table/nan_col_series.tex")
nan_cols_df.head(20)

There are 414 columns with missing values (0.9539 of all columns)


,0,1
id_24,585793,0.991962
id_25,585408,0.991310
id_07,585385,0.991271
id_08,585385,0.991271
id_21,585381,0.991264
id_26,585377,0.991257
id_27,585371,0.991247
id_23,585371,0.991247
id_22,585371,0.991247
dist2,552913,0.936284


In [39]:
# # Vérifier parmi les valeurs manquantes si celles-ci sont exclusivement associées à des transactions
# # légitimes ou frauduleuses
# data = defaultdict()
# for col in nan_cols_ratio_series[nan_cols_ratio_series > 0.].index:
#     data[col] = {"isFraud==0": 0, "isFraud==1": 0}
#     data[col]["isFraud==0"] = df[df["isFraud"] == 0][col].isna().sum()
#     data[col]["isFraud==1"] = df[df["isFraud"] == 1][col].isna().sum()

# nan_count_per_cls = pd.DataFrame(data)
# nan_count_per_cls.to_csv("export/table/is_fraud_count_nan_cols.csv")
# nan_count_per_cls

In [40]:
# Colonnes avec 1 ou plusieurs valeurs nulles
nan_cols = df.columns[df.isna().sum() > 0]
# Colonnes ayant le même nombre de valeurs manquantes
dup_cols = df[nan_cols].columns[[df[nan_cols].isna().sum().duplicated()]]
# Colonnes obtenues à l'étape précédente dont le nombre de valeurs négatives ne dépasse pas 1% des données
cols = df[dup_cols].columns[df[dup_cols].isna().sum() < (.01 * len(df))]
# Suppression des valeurs manquantes
len_before = len(df)
df.dropna(inplace=True, subset=cols)
# Validation
assert df[cols].isna().sum().sum() == 0., "Il reste des valeurs manquantes"
assert df.isna().sum().sum() > 0., "Mauvaise assignation: il devrait rester des valeurs manquantes dans le reste des données"
diff = len_before - len(df)
print("Rows deleted: {}\nRatio: {:2.4f}".format(diff, diff / len_before))

C:\Users\verdi\.conda\envs\anomaly_detection\lib\site-packages\pandas\core\indexes\base.py:5055: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


Rows deleted: 1441
Ratio: 0.0024


In [41]:
# Affichage des colonnes dont le ratio de valeurs manquantes dépassent 50% des données
cols_above_thresh = df.columns[(df.isna().sum() / len(df)) > 0.5]
cols_above_thresh

Index(['dist1', 'dist2', 'R_emaildomain', 'D5', 'D6', 'D7', 'D8', 'D9', 'D12',
       'D13',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=214)

In [42]:
# Fonction grandement inspirée de https://www.kaggle.com/code/davidcairuz/feature-engineering-lightgbm
def id_split(dataframe):
    dataframe['device_name'] = dataframe['DeviceInfo'].str.split('/', expand=True)[0].str.split(' ', expand=True)[0]
    dataframe.loc[dataframe['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG'
    dataframe.loc[dataframe['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV'
    dataframe.loc[dataframe['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux'
    dataframe.loc[dataframe['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony'
    dataframe.loc[dataframe['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC'
    dataframe.loc[dataframe['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'

    dataframe.loc[dataframe.device_name.isin(dataframe.device_name.value_counts()[dataframe.device_name.value_counts() < 200].index), 'device_name'] = "Others"
    
    return dataframe

In [43]:
# Conversion de la colonne "DeviceInfo" vers le nouvel attribut "device_name"
df = id_split(df)
df["device_name"] = df["device_name"].fillna("NoDevice")
# Retrait des colonnes "DeviceInfo" et "DeviceType" qui deviennent redondantes
df.drop(["DeviceInfo", "DeviceType"], axis=1, inplace=True)

In [44]:
def mean_target_encoding(dataframe, column, na_value):
    dataframe[column] = dataframe[column].fillna(na_value)
    mean_enc = dataframe.groupby(column)["isFraud"].mean()
    dataframe.loc[:, column] = dataframe.loc[:, column].map(mean_enc)
    return dataframe

In [45]:
cols = ["device_name", "P_emaildomain", "R_emaildomain", "id_31"]
na_vals = ["NoDevice", "NoEmail", "NoEmail", "NoBrowser"]
for c, v in zip(cols, na_vals):
    df = mean_target_encoding(df, c, v)
    assert df[c].isna().sum() == 0, "La colonne '%s' contient toujours des valeurs manquantes" % c

In [68]:
cols_above_thresh = df.columns[(df.isna().sum() / len(df)) >= 0.5]
# f[['id_28', 'id_29', 'id_30', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36','id_37', 'id_38']
cols_above_thresh

Index(['dist1', 'dist2', 'D5', 'D6', 'D7', 'D8', 'D9', 'D12', 'D13', 'D14',
       ...
       'id_28', 'id_29', 'id_30', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36',
       'id_37', 'id_38'],
      dtype='object', length=210)

In [81]:
# Affichage des colonnes dont le ratio de valeurs manquantes représente moins de 50% des données 
# (et qui contiennent au moins une valeur manquante)
cols_below_thresh = df.columns[(df.isna().sum() / len(df)) < 0.5]
cols_below_thresh = df[cols_below_thresh].columns[df[cols_below_thresh].isna().sum() > 0]


In [91]:
df[cols_below_thresh].select_dtypes(include=np.number)


,card2,card3,card5,addr1,addr2,D2,D3,D4,D10,D11,...,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94
0,NaN,150.0,142.0,315.0,87.0,NaN,13.0,NaN,13.0,13.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,404.0,150.0,102.0,325.0,87.0,NaN,NaN,0.0,0.0,NaN,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,490.0,150.0,166.0,330.0,87.0,NaN,NaN,0.0,0.0,315.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,567.0,150.0,117.0,476.0,87.0,112.0,0.0,94.0,84.0,NaN,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,514.0,150.0,102.0,420.0,87.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,NaN,150.0,226.0,272.0,87.0,29.0,30.0,NaN,56.0,56.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
590536,225.0,150.0,224.0,204.0,87.0,NaN,NaN,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
590537,595.0,150.0,224.0,231.0,87.0,NaN,NaN,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
590538,481.0,150.0,224.0,387.0,87.0,22.0,0.0,22.0,22.0,22.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
from impyute.imputation.cs import fast_knn
# sys.setrecursionlimit(100000) #Increase the recursion limit of the OS
fast_knn(df[cols_below_thresh].select_dtypes(include=np.number).values, k=30)

KeyboardInterrupt: 

## Unique values

In [48]:
num_df.columns[num_df.nunique() == 1]


Index([], dtype='object')